In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import shuffle
import numpy as np
import tensorflow as tf

In [2]:
M = 2
K = 3
n = 100
N = n * K

In [3]:
X1 = np.random.randn(n, M) + np.array([0, 10])
X2 = np.random.randn(n, M) + np.array([5, 5])
X3 = np.random.randn(n, M) + np.array([10, 0])
Y1 = np.array([[1, 0, 0] for i in range(n)])
Y2 = np.array([[0, 1, 0] for i in range(n)])
Y3 = np.array([[0, 0, 1] for i in range(n)])

X = np.concatenate((X1, X2, X3), axis=0)
Y = np.concatenate((Y1, Y2, Y3), axis=0)

In [4]:
W = tf.Variable(tf.zeros([M, K]))
b = tf.Variable(tf.zeros([K]))

x = tf.placeholder(tf.float32, shape=[None, M])
t = tf.placeholder(tf.float32, shape=[None, K])
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [5]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y), reduction_indices=[1])) 

In [6]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [7]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))

In [8]:
batch_size = 50
n_batches = N // batch_size

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(20):
    X_, Y_ = shuffle(X, Y)
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={
            x:X_[start:end],
            t:Y_[start:end]
        })

In [10]:
X_, Y_ = shuffle(X, Y)

classified = correct_prediction.eval(session=sess, feed_dict={x: X_[0:10], t: Y_[0:10]})
prob = y.eval(session=sess, feed_dict={x: X_[0:10]})
print('classified: \n', classified)
print()
print('output probability: \n', prob)

classified: 
 [ True  True  True  True  True  True  True  True  True  True]

output probability: 
 [[  5.64577756e-03   9.88054812e-01   6.29938953e-03]
 [  1.44765392e-01   8.54136705e-01   1.09797483e-03]
 [  9.86593068e-01   1.34069324e-02   5.86681459e-09]
 [  9.58921191e-08   5.92853129e-02   9.40714598e-01]
 [  9.95771945e-01   4.22804477e-03   6.24577812e-09]
 [  9.97860610e-01   2.13935087e-03   1.69078707e-08]
 [  1.87107130e-09   4.55825496e-03   9.95441675e-01]
 [  2.02955622e-02   9.59287643e-01   2.04168297e-02]
 [  2.52857688e-03   8.14478874e-01   1.82992563e-01]
 [  1.48566288e-08   1.33185545e-02   9.86681461e-01]]


## Keras

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [12]:
model = Sequential()
model.add(Dense(input_dim=M, units=K))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1))

In [13]:
minibatch_size=50
model.fit(X, Y, epochs=20, batch_size=minibatch_size)

Epoch 1/20
300/300 [==============================] - 0s 178us/step - loss: 3.0767
Epoch 2/20
300/300 [==============================] - 0s 32us/step - loss: 0.2709
Epoch 3/20
300/300 [==============================] - 0s 21us/step - loss: 0.1659
Epoch 4/20
300/300 [==============================] - 0s 22us/step - loss: 0.1263
Epoch 5/20
300/300 [==============================] - 0s 20us/step - loss: 0.1034
Epoch 6/20
300/300 [==============================] - 0s 33us/step - loss: 0.0878
Epoch 7/20
300/300 [==============================] - 0s 20us/step - loss: 0.0772
Epoch 8/20
300/300 [==============================] - 0s 18us/step - loss: 0.0693
Epoch 9/20
300/300 [==============================] - 0s 20us/step - loss: 0.0633
Epoch 10/20
300/300 [==============================] - 0s 18us/step - loss: 0.0577
Epoch 11/20
300/300 [==============================] - 0s 20us/step - loss: 0.0530
Epoch 12/20
300/300 [==============================] - 0s 18us/step - loss: 0.0494
Epoch 13/20


In [14]:
X_, Y_ = shuffle(X, Y)
classes = model.predict_classes(X_[0:10], batch_size=minibatch_size)
prob=model.predict_proba(X_[0:10], batch_size=1)
print('classified: \n', np.argmax(model.predict(X_[0:10]), axis=1) == classes)
print()
print('output probability: \n', prob)

10/10 [==============================] - 0s 447us/step
classified: 
 [ True  True  True  True  True  True  True  True  True  True]

output probability: 
 [[  9.08883351e-08   1.06551550e-01   8.93448353e-01]
 [  6.56254683e-03   9.49420452e-01   4.40170243e-02]
 [  1.18937997e-08   2.86523253e-02   9.71347690e-01]
 [  9.80829835e-01   1.91701204e-02   1.57659894e-08]
 [  9.49359715e-01   5.06401211e-02   1.57890597e-07]
 [  1.58135137e-07   6.64269999e-02   9.33572888e-01]
 [  8.85825013e-09   7.54037593e-03   9.92459595e-01]
 [  8.52790549e-09   2.50489730e-02   9.74950969e-01]
 [  7.68515542e-02   9.20742512e-01   2.40591890e-03]
 [  9.20356035e-01   7.96428025e-02   1.17799198e-06]]
